In [1]:
import pandas as pd
import configparser
import psycopg2
import os

In [ ]:
# move up one directory for config files
os.chdir('..')
os.getcwd()

In [ ]:
config = configparser.ConfigParser()
config.read('dwh.cfg')
    
host = config['CLUSTER']['DB_ENDPOINT']
dbname = config['CLUSTER']['DB_NAME']
user = config['CLUSTER']['DB_USER']
password = config['CLUSTER']['DB_PASSWORD']
port = config['CLUSTER']['DB_PORT']

conn = psycopg2.connect("host={} dbname={} user={} password={} port={}".format(host, dbname, user, password, port))
cur = conn.cursor()

In [ ]:
table_names = ['dim_user', 'dim_song', 'dim_artist', 'dim_time', 'fact_sonplay']

In [ ]:
# Create Row Count Check Queries

user_table_count = ('''
SELECT Count(*)
FROM dim_user
;
''')

song_table_count = ('''
SELECT Count(*)
FROM dim_song
;
''')

artist_table_count =('''
SELECT Count(*)
FROM dim_artist
;
''')

time_table_count = ('''
SELECT Count(*)
FROM dim_time
;
''')


fact_table_count = ('''
SELECT Count(*)
FROM fact_songplay
;
''')

In [ ]:
# Execute Row Count Check Queries

count_rows_queries = [user_table_count, song_table_count, artist_table_count, time_table_count]
for i in range(len(count_rows_queries)):
        print("Check Row Count Table {}".format(table_names[i]))
        cur.execute(query[i])
        conn.commit()

In [ ]:
# Create Duplication Check Queries

user_table_dups = ('''
SELECT user_id, first_name, last_name, level, COUNT(*)
FROM sparkify.dim_user
GROUP BY user_id, first_name, last_name, level
ORDER BY COUNT(*) DESC
LIMIT 5
;
''')

song_table_dups = ('''
SELECT
    s.song_id,
    s.title,
    COUNT(*)
FROM dim_song s
GROUP BY s.song_id, s.title
ORDER BY COUNT(*) DESC
LIMIT 5;
''')

artist_table_dups =('''
SELECT
    a.artist_id,
    a.name,
    COUNT(*)
FROM dim_artist a
GROUP BY a.artist_id, a.name
ORDER BY COUNT(*) DESC
LIMIT 5;
''')

time_table_dups = ('''
SELECT
    t.time_key,
    COUNT(*)
FROM dim_time t
GROUP BY t.time_key
ORDER BY COUNT(*) DESC
LIMIT 5;
''')



In [2]:
# Execute Duplication Check Queries

# Function
## Inputs a query, sql connection and table name
## Ouputs if query producted duplicates or not with statement
def check_duplicates(query, conn, table):

    count_df = pd.read_sql(query, conn)
    count_s = count_df['count']
    total_count = 0
    for val in count_s:
        if val == 1:
            total_count += 0
        
    if total_count > 0:
        return(print("Rows duplicated in table {}".format(table)))
        
    else:
        return(print("No rows duplicated in table {}".format(table)))
    

check_duplicate_queries = [user_table_dups, song_table_dups, artist_table_dups, time_table_dups]

duplicate_check_queries = [user_table_count, song_table_count, artist_table_count, time_table_count]
for i in range(len(duplicate_check_queries)):
        print("Check Row Count Table {}".format(table_names[i]))
        cur.execute(query[i])
        conn.commit()


NameError: name 'user_table_dups' is not defined